[View in Colaboratory](https://colab.research.google.com/github/luciaedw/SUTD_Blockchain/blob/steve/BLKA2Q1.ipynb)

In [3]:
!pip install ecdsa

    100% |████████████████████████████████| 92kB 3.2MB/s 


In [0]:
import ecdsa
import json
from binascii import hexlify, unhexlify
import hashlib
import time
import random

class Transaction:
    
    def __init__(self, sender, receiver, amount, signature, comment=""):
        self.sender = sender
        self.receiver = receiver
        self.amount = amount
        self.signature = signature
        self.comment = comment
    
    # Method for making a new transaction using input fields
    @classmethod
    def new(cls, sender, receiver, amount, signature, comment=""):
      if type(sender) is str:
        sender = sender
      if type(sender) is bytes:
        sender = hexlify(sender).decode()
      elif type(sender) is ecdsa.keys.VerifyingKey:
        sender = hexlify(sender.to_string()).decode()
      return cls(sender, receiver, amount, signature, comment)
    
    # Print fields for debugging
    def printSelf(self):
      print(self.sender)
      print(self.receiver)
      print(self.amount)
      print(self.signature)
      print(self.comment)
    
    # Serializes object to JSON string
    def to_json(self):
        return json.dumps(self.__dict__)
    
    # Instantiates/Deserializes object from JSON string
    @classmethod
    def from_json(cls, inpStr):   
        jsonData = json.loads(inpStr)
        jsonKeys = list(jsonData.keys())
        if('sender' not in jsonKeys or 'receiver' not in jsonKeys or 'amount' not in jsonKeys or 'signature' not in jsonKeys):
          raise ValueError('Missing keys in Json object, expecting: sender, receiver, amount, signature')
        else:
          if('comment' in jsonKeys):
            comment = jsonData['comment']
          else:
            comment = ''
          return cls(jsonData['sender'], jsonData['receiver'], jsonData['amount'], jsonData['signature'], comment)
 
    #Sign object, takes private key as input
    def sign(self, privKey):
        headerDict = dict(self.__dict__)
        del headerDict['signature']
        headerJson = json.dumps(headerDict, sort_keys=True)
        header = str(headerJson).encode()
        signature = privKey.sign(header)
        self.signature = hexlify(signature).decode()
        return signature

    #Validate signature against expected results
    def validate(self):
        signature = unhexlify(self.signature.encode())
        headerDict = dict(self.__dict__)
        del headerDict['signature']
        headerJson = json.dumps(headerDict, sort_keys = True)
        header = str(headerJson).encode()
        vk = ecdsa.VerifyingKey.from_string(unhexlify(self.sender.encode()))
        return(vk.verify(signature, header))

    #Checks of transactions are the same
    def __eq__(self, trans):
      mySig = unhexlify(self.__dict__['signature'].encode())
      transSig = unhexlify(json.loads(trans.to_json())['signature'].encode())
      return mySig == transSig


In [0]:
# Class for nodes in merkle tree, keeps track of parents, children, and height
class merkleNode:
  def __init__(self, leftChild, rightChild, hashVal):
    self.parent = None
    self.leftChild = leftChild
    self.rightChild = rightChild
    self.hashVal = hashVal
    if(leftChild == None):
      self.height = 0
    else:
      self.height = leftChild.getHeight() + 1
  
  def getHeight(self):
    return self.height
  
  def getHash(self):
    return self.hashVal
  
  def setParent(self, parentNode):
    self.parent = parentNode
   
  def getParent(self):
    return self.parent
  
  
# Merkle tree implementation, keeps track of nodes(in a dictionary), height, entries(items to be added)
# rootNode, and the root hash
class merkleTree:
  def __init__(self):
    self.nodes = dict()
    self.height = 0
    self.entries = []
    self.rootNode = None
    self.rootHash = ''
    
  def getRoot(self):
    return self.rootNode
  
  def getRootHash(self):
    return self.rootHash
  
  # Additem to entries, a list of all items used to build tree later
  def add(self, transaction):
    self.entries.append(transaction)
  
  # Builds a merkle tree using items from entries
  def build(self):
    self.nodes = dict()
    stack = list(self.entries)
    self.height = 0
    if(len(stack) == 0):
      return 0
        
    else:
      for i in range(len(stack)):
        item = stack.pop(0)
        itemHash = hashlib.sha512(('0'+item.to_json()).encode()).hexdigest()

        itemNode = merkleNode(None, None, itemHash)
        self.nodes[itemHash] = itemNode
        stack.append(itemNode)

    while(len(stack)>1):
      leftNode = stack.pop(0)
      rightNode = stack.pop(0)
      newHash = hashlib.sha512(('1' + leftNode.getHash() + rightNode.getHash()).encode()).hexdigest()
      newNode = merkleNode(leftNode, rightNode, newHash)
      stack.append(newNode)
      self.nodes[newHash] = newNode
      leftNode.setParent(newNode)
      rightNode.setParent(newNode)
    self.rootNode = stack[0]
    self.rootHash = stack[0].getHash()
    return self.rootHash
  
  # Gets a proof as a 2d list, the first element of each index represents which
  # side the hash is from
  def getProof(self, transaction):
    proof = []
    transHash = hashlib.sha512(('0'+transaction.to_json()).encode()).hexdigest()
    if transHash in self.nodes.keys():
      transNode = self.nodes[transHash]
    else:
      return proof
    while(transNode is not self.rootNode):
      if(transNode.getParent().leftChild is transNode):
        proof.append(['right', transNode.getParent().rightChild.getHash()])
      else:
        proof.append(['left', transNode.getParent().leftChild.getHash()])
      transNode = transNode.getParent()
    return(proof)
  
  # Verify proof against root
  def verifyProof(self, transaction, proof, root):
    transHash = hashlib.sha512(('0'+transaction.to_json()).encode()).hexdigest()
    nextHash = transHash
    while(len(proof)>0):
      partnerHash = proof.pop(0)
      if partnerHash[0] == 'left':
        nextHash = hashlib.sha512(("1" + partnerHash[1] + nextHash).encode()).hexdigest()
      else:
        nextHash = hashlib.sha512(("1" + nextHash + partnerHash[1]).encode()).hexdigest()
                                  
    if(nextHash == root):
      return True
    else:
      return False
    
  def validate(self):
    for transaction in self.entries:
      proof = self.getProof(transaction)
      if not self.verifyProof(transaction, proof, self.rootHash) or not transaction.validate():
        return False
    return True
  

In [0]:
class block:
  def __init__(self, prev, prevHead, root, tree):
    self.prev = prev
    self.prevHead = prevHead
    self.root = root
    self.tree = tree
    self.time = int(time.time())
    self.nonce = 0
    self.hash = 0
    self.hash = self.hashSelf()

  def hashSelf(self):
    headerDict = dict(self.__dict__)
    del headerDict['prev']
    del headerDict['tree']
    del headerDict['hash']
    jsonTrans = json.dumps(headerDict)

    return hashlib.sha512(jsonTrans.encode()).hexdigest()
  
  def getHash(self):
    return self.hash
  
  def reHash(self):
    self.nonce += 1
    self.hash=self.hashSelf()
    return self.hash
  
  def validate(self):
    headerDict = dict(self.__dict__)
    del headerDict['prev']
    del headerDict['tree']
    del headerDict['hash']
    jsonTrans = json.dumps(headerDict)
    if self.tree:
      return(self.hash == hashlib.sha512(jsonTrans.encode()).hexdigest() and self.tree.validate())
    else:
      return(self.hash == hashlib.sha512(jsonTrans.encode()).hexdigest())
    
    
    
class blockChain:
  def __init__(self):
    self.genesis = block(None, None, '0', None)
    self.end = self.genesis
    self.blocks = [self.genesis]
    self.globalParam = int('00000fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff', 16)
    
  def add(self, treeRoot, tree):
    newBlock = block(self.end, self.end.getHash(), treeRoot, tree)
    while(True):
      if(int(newBlock.getHash(), 16) <= self.globalParam):
        self.end = newBlock
        self.blocks.append(newBlock)
        break
      else:
        newBlock.reHash()
    print("added block")
    print(newBlock.getHash())
    
    
  def validate(self):
    for block in self.blocks:
      
      if not block.validate() or not (int(block.getHash(), 16) <= self.globalParam):
        if(block != self.genesis):
          return False
    return True

In [0]:
# Testing Merkle Tree Stuff
from ecdsa import SigningKey

print("building tree")
trannies = []
for i in range(random.randint(100,1000)):
  sk = SigningKey.generate() # uses NIST192p
  vk = sk.get_verifying_key().to_string()
  newTrans = Transaction.new(hexlify(vk).decode(),'b',random.randint(100,1000),'a',i)
  newTrans.sign(sk)
  trannies.append(newTrans)
  if i%100 == 0:
    print('built %d entries'%(i))

testTree = merkleTree()

for t in trannies:
  testTree.add(t)

  
print('done building')
testTree.build()


testProof = testTree.getProof(trannies[0])
print(testTree.verifyProof(trannies[0], testProof, testTree.getRoot().getHash()))



sk = SigningKey.generate() # uses NIST192p
vk = sk.get_verifying_key().to_string()
newTrans = Transaction.new(hexlify(vk).decode(),'b',random.randint(50,100),'a',i)
newTrans.sign(sk)

print(testTree.verifyProof(newTrans, testProof, testTree.getRoot().getHash()))

failProof = testTree.getProof(newTrans)
print(failProof)

for i in range (10):
  index = random.randint(0, len(trannies)-1)
  testProof = testTree.getProof(trannies[index])
  print(testTree.verifyProof(trannies[index], testProof, testTree.getRoot().getHash()))

In [61]:
# Testing Blockchain Stuff
from ecdsa import SigningKey

testChain = blockChain()

for i in range(5):
  print("building tree")
  trannies = []
  for i in range(random.randint(10,100)):
    sk = SigningKey.generate() # uses NIST192p
    vk = sk.get_verifying_key().to_string()
    newTrans = Transaction.new(hexlify(vk).decode(),'b',random.randint(100,1000),'a',i)
    newTrans.sign(sk)
    trannies.append(newTrans)
    if i%100 == 0:
      print('built %d entries'%(i))

  testTree = merkleTree()

  for t in trannies:
    testTree.add(t)

  testTree.build()
  print('done building')
  print('adding block')
  
  newRootHash = testTree.getRootHash()
  testChain.add(newRootHash, testTree)
  
  
print(testChain.genesis.validate())
print(testChain.genesis.root)
print(testChain.validate())


building tree
built 0 entries
done building
adding block
added block
000007f964123549367b9df286b564cf0c9437ea1bbc389b90410278d7420b669cf29fb2fc4b836fec34b2ec40adc2495cc0bbbe1e3f5c47ce136dd56c915885
building tree
built 0 entries
done building
adding block
added block
000003fc01e68f0f85263112535570bd68619cfc657faaa69f55aa06f8226c2aa825703134fc933b551e4db14416dfe2445bba48a31329f232c66106d9942cce
building tree
built 0 entries
done building
adding block
added block
000007ae834f007325847cb1e11235d88bdb07f58bde9c076857bc318f213899c09dd1081182f30b25ca5edb8366544de843bd51d6325bbfcb94f9c882123499
building tree
built 0 entries
done building
adding block
added block
000001190409b9835d7238ed1ac950007b1f608b35fb87d06178b08611df8efe98fccda8a1c7876bad88dcbd35cca506ee7d3b5e6c56a66ce5f4efcf8a842932
building tree
built 0 entries
done building
adding block
added block
00000b52a92584ea92d2499599cdd642af187bde614814b7421005a825d3ae7d7253e046585eeb1e87332501f465b435bd8f4c77fcb6b9c9cdca3bdc37445753
True
0
Tru